# Exampe 4.1: Load effects in three story shear frame
In this example we will study the shear force and bending moment in one of the columns between degree of freedom 1 and 2 in a three story shear frame.

<img src=".\figures\Fig_4_1_.svg">

![Alt text](.\figures\Fig_4_1_.svg)